In [13]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from data import DataFrame
from utils import open_csv_file
from mlxtend.plotting import plot_confusion_matrix

import sklearn
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings('ignore')

In [14]:
training_data = open_csv_file('data_sample_one.csv')
x = DataFrame(training_data)
training_data_y = open_csv_file('data.csv')


In [15]:
x.remove_columns(['age', 'time'])
X = x.get_df()[['time_bucket', 'ejection_fraction', 'serum_creatinine']]
y = DataFrame(training_data_y).get_df()['DEATH_EVENT']
result = pd.concat([X, y], axis=1)



In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

nromal data

In [17]:
new_testing_data = pd.concat([X_test, y_test], axis=1)
new_testing_data.to_csv('v2_testing_data.csv', index=False)

In [18]:
new_training_data = pd.concat([X_train, y_train], axis=1)
new_training_data.to_csv('v2_training_data.csv', index=False)

sampling data


In [19]:
from sklearn.utils import resample
df_majority = new_training_data[(new_training_data['DEATH_EVENT'] == 1)]
df_minority = new_training_data[(new_training_data['DEATH_EVENT'] == 0)]
majority_count = df_majority['DEATH_EVENT'].value_counts().astype(int)

majority_count_value = majority_count.values[0]
df_minority_upsampled = resample(df_minority, replace=True, n_samples=majority_count_value, random_state=42)
df_upsampled = pd.concat([df_minority_upsampled, df_majority])
death = df_upsampled['DEATH_EVENT']

In [20]:
df_upsampled.to_csv('v2_training_sampling_data.csv', index=False)

SMOTE 

In [21]:
pip install imbalanced-learn


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [22]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(sampling_strategy="minority", random_state=42)
oversampled_X, oversampled_Y = sm.fit_resample(new_testing_data.drop('DEATH_EVENT', axis=1), new_testing_data['DEATH_EVENT'])
df_upsampled_v2 = pd.concat([pd.DataFrame(oversampled_Y), pd.DataFrame(oversampled_X)], axis=1)
df_upsampled_v2['DEATH_EVENT']
df_upsampled_v2.to_csv('v2_training_smote_data.csv', index=False)